# SciPy概述

NumPy替我们搞定了向量和矩阵的相关操作，基本上算是一个高级的科学计算器。SciPy基于NumPy提供了更为丰富和高级的功能扩展，在统计、优化、插值、数值积分、时频转换等方面提供了大量的可用函数，基本覆盖了基础科学计算相关的问题。

In [2]:
import numpy as np
import scipy.stats as stats # 统计
import scipy.optimize as opt # 优化

## 统计

### 生成随机数

生成n个随机数可用`rv_continuous.rvs(size=n)`或`rv_discrete.rvs(size=n)`

- `rv_continuous`表示连续型的随机分布，如均匀分布（`uniform`）、正态分布（`norm`）、贝塔分布（`beta`）等；
- `rv_discrete`表示离散型的随机分布，如伯努利分布（`bernoulli`）、几何分布（`geom`）、泊松分布（`poisson`）等。

点击查阅 [统计函数 (scipy.stats) 手册](https://docs.scipy.org.cn/doc/scipy/reference/stats.html) 或 [英文版](https://docs.scipy.org/doc/scipy/reference/stats.html)

若在SciPy没有满足需求，可以采用[StatsModels](http://statsmodels.sourceforge.net)（提供了更为专业，更多的统计相关函数）。

In [10]:
rv_unif = stats.uniform.rvs(size=10)
print(rv_unif)

[0.9357919  0.39175694 0.78447367 0.56149997 0.9892763  0.88851423
 0.42201385 0.9239862  0.43197438 0.2977279 ]


In [11]:
rv_beta = stats.beta.rvs(size=10, a=4, b=2)
print(rv_beta)

[0.74623507 0.76359656 0.39081388 0.53724531 0.50424926 0.75267061
 0.70829427 0.64356349 0.44675237 0.78371563]


一些优化做法：

- `SciPy.stats`支持定义出某个具体的分布的对象，我们可以做如下的定义，让`beta`直接指代具体参数`a=4`和`b=2`的贝塔分布。
- 指定了随机数的生成种子

In [13]:
np.random.seed(seed=2025)
rv_beta = stats.beta.rvs(size=10, a=4, b=2)
print("method 1:")
print(rv_beta)

method 1:
[0.55452864 0.3571434  0.83274347 0.73264615 0.5621855  0.93875378
 0.7806391  0.84170717 0.55878702 0.80312908]


In [14]:
np.random.seed(seed=2025)
beta = stats.beta(a=4, b=2)
print("method 2:")
print(beta.rvs(size=10))

method 2:
[0.55452864 0.3571434  0.83274347 0.73264615 0.5621855  0.93875378
 0.7806391  0.84170717 0.55878702 0.80312908]


### 假设检验

生成一组数据，并查看相关的统计量。

`norm` 一个正态连续随机变量。位置 (`loc`) 关键字指定均值。 尺度 (`scale`) 关键字指定标准差。

In [19]:
np.random.seed(seed=2024)

norm_dist = stats.norm(loc=0.5, scale=2)
n = 200
dat = norm_dist.rvs(size=n)
print("mean of data is: " + str(np.mean(dat)))
print("median of data is: " + str(np.median(dat)))
print("standard deviation of data is: " + str(np.std(dat)))

mean of data is: 0.6144130303808365
median of data is: 0.5973383111601445
standard deviation of data is: 2.024935628110048


最简单的是检验这一组数据是否服从假设的分布，如正态分布。

典型的单样本假设检验问题，最为常见的解决方案是采用K-S检验（ Kolmogorov-Smirnov test）。

单样本K-S检验的原假设是给定的数据来自和原假设分布相同的分布，在SciPy中提供了kstest函数，参数分别是数据、拟检验的分布名称和对应的参数。

In [20]:
mu = np.mean(dat)
sigma = np.std(dat)
stat_val, p_val = stats.kstest(dat, 'norm', (mu, sigma))
print('KS-statistic D = %6.3f p-value = %6.4f' % (stat_val, p_val))

KS-statistic D =  0.032 p-value = 0.9834


假设检验的`p-value`值很大（在原假设下，p-value是服从[0, 1]区间上的均匀分布的随机变量，可参考 [维基百科 P-value 词条](http://en.wikipedia.org/wiki/P-value) ），因此我们接受原假设，即该数据通过了正态性的检验。

在正态性的前提下，我们可进一步检验这组数据的均值是不是0。典型的方法是`t`检验（`t-test`）。

单样本的`t`检验函数为`ttest_1samp`。

In [21]:
stat_val, p_val = stats.ttest_1samp(dat, 0)
print('One-sample t-statistic D = %6.3f, p-value = %6.4f' % (stat_val, p_val))

One-sample t-statistic D =  4.280, p-value = 0.0000


看到`p-value<0.05`，即给定显著性水平`0.05`的前提下，我们应拒绝原假设：数据的均值为`0`。

再生成一组数据，尝试一下双样本的`t`检验（`ttest_ind`）。

In [25]:
norm_dist2 = stats.norm(loc=-0.2, scale=1.2)
dat2 = norm_dist2.rvs(size=(n-19)) # 随便取长度，和 dat1 样本大小、方差均不相等
stat_val, p_val = stats.ttest_ind(dat, dat2, equal_var=False)
print('Two-sample t-statistic D = %6.3f, p-value = %6.4f' % (stat_val, p_val))

Two-sample t-statistic D =  4.429, p-value = 0.0000


第二组数据样本大小、方差和第一组均不相等，在运用`t`检验时需要使用`Welch's t-test`，即指定`ttest_ind`中的`equal_var=False`。得到了比较小的p-value$，在显著性水平0.05的前提下拒绝原假设，即认为两组数据均值不等。

得到了比较小的`p-value`，在显著性水平`0.05`的前提下拒绝原假设，即认为两组数据均值不等。

`stats`还提供其他大量的假设检验函数，如`bartlett`和`levene`用于检验方差是否相等；`anderson_ksamp`用于进行`Anderson-Darling`的`K-样本`检验等。

### 其他函数

#### 分布与分位

需要知道某数值在一个分布中的分位，或者给定了一个分布，求某分位上的数值。这可以通过`cdf`和`ppf`函数完成。

In [32]:
g_dist = stats.gamma(a=2)
print("quantiles of 2, 4 and 5:")
print(g_dist.cdf([2, 4, 5]))
print("Values of 25%, 50% and 90%:")
print(g_dist.ppf([0.25, 0.5, 0.95]))
print("Values of 25%, 50% and 90%:")
print(g_dist.pdf([0.25, 0.5, 0.95]))

quantiles of 2, 4 and 5:
[0.59399415 0.90842181 0.95957232]
Values of 25%, 50% and 90%:
[0.96127876 1.67834699 4.74386452]
Values of 25%, 50% and 90%:
[0.1947002  0.30326533 0.36740397]


对于一个给定的分布，可以用moment很方便的查看分布的矩信息，例如我们查看N(0,1)的六阶原点矩

In [34]:
stats.norm.moment(6, loc=0, scale=1)

np.float64(15.000000000000004)

#### 统计描述
`describe`函数提供对数据集的统计描述分析，包括数据样本大小，极值，均值，方差，偏度和峰度：

In [35]:
norm_dist = stats.norm(loc=0, scale=1.8)
dat = norm_dist.rvs(size=100)
info = stats.describe(dat)
print("Data size is: " + str(info[0]))
print("Minimum value is: " + str(info[1][0]))
print("Maximum value is: " + str(info[1][1]))
print("Arithmetic mean is: " + str(info[2]))
print("Unbiased variance is: " + str(info[3]))
print("Biased skewness is: " + str(info[4]))
print("Biased kurtosis is: " + str(info[5]))

Data size is: 100
Minimum value is: -6.611219384688499
Maximum value is: 4.59998935584996
Arithmetic mean is: -0.010790196944001471
Unbiased variance is: 3.334150803038929
Biased skewness is: -0.33336530030397216
Biased kurtosis is: 1.0054260326043316


#### 分布估计（极大似然估计）
当我们知道一组数据服从某些分布的时候，可以调用`fit`函数来得到对应分布参数的极大似然估计（MLE, maximum-likelihood estimation）。

In [2]:
norm_dist = stats.norm(loc=0, scale=1.8)
dat = norm_dist.rvs(size=100)
mu, sigma = stats.norm.fit(dat)
print ("MLE of data mean:" + str(mu))
print ("MLE of data standard deviation:" + str(sigma))

MLE of data mean:0.2573138478530397
MLE of data standard deviation:1.9508261218529632


#### 相关系数
`pearsonr`和`spearmanr`可以计算`Pearson`和`Spearman`相关系数，这两个相关系数度量了两组数据的相互线性关联程度.

- Pearson 相关系数 (皮尔逊相关系数)：衡量两个连续变量之间的‌线性相关程度‌
- Spearman 相关系数 (斯皮尔曼相关系数)：衡量两个变量之间的‌单调相关程度‌(无论线性与否)

| 关系类型 | Pearson | Spearman |
| --- | --- | --- |
| 线性关系 | 高 | 高 |
| 指数增长关系 | 低 | 高 |
| 二次函数关系 | 低 | 低 |
| 完全随机关系 | ≈0 | ≈0 |
| 异常值影响 | 敏感 | 不敏感 |

使用方式：
- 若关注严格的‌线性关系‌ → 选择 ‌Pearson‌
- 若关注‌单调趋势‌(包含非线性) → 选择 ‌Spearman‌
- 当数据含异常值或非正态分布时 → 优先使用 ‌Spearman‌

其中的p-value表示原假设（两组数据不相关）下，相关系数的显著性。

In [5]:
norm_dist = stats.norm()
dat1 = norm_dist.rvs(size=100)

exp_dist = stats.expon()
dat2 = exp_dist.rvs(size=100)

cor, pval = stats.pearsonr(dat1, dat2)
print("Pearson correlation coefficient: " + str(cor))

cor, pval = stats.spearmanr(dat1, dat2)
print("Spearman's rank correlation coefficient: " + str(cor))

Pearson correlation coefficient: -0.015581389157782885
Spearman's rank correlation coefficient: -0.0069846984698469835


#### 线性回归

In [7]:
x = stats.chi2.rvs(3, size=50)
y = 2.5 + 1.2 * x + stats.norm.rvs(size=50, loc=0, scale=1.5)
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
print("Slope of fitted model is:" , slope)
print("Intercept of fitted model is:", intercept)
print("R-squared:", r_value**2)

Slope of fitted model is: 1.1248923158457005
Intercept of fitted model is: 2.717408206454295
R-squared: 0.6786187608427824


## 优化

优化问题在投资中可谓是根本问题，如果手上有众多可选的策略，应如何从中选择一个“最好”的策略进行投资呢？这时就需要用到一些优化技术针对给定的指标进行寻优。随着越来越多金融数据的出现，机器学习逐渐应用在投资领域，在机器学习中，优化也是十分重要的一个部分。

所谓的无约束优化问题指的是一个优化问题的最优可行集合是目标函数自变量的定义域，即没有外部的限制条件。

例如，$f(x)=x^2-4.8x+1.2$ 求解$f(x)$的最小值，就是一个无约束优化问题。

而求解$f(x)=x^2-4.8x+1.2$在$X >= 0$条件下的最小值，则是一个带约束的优化问题。

### 无约束优化

进一步假设问题全部是凸优化问题（参考教材[斯坦福大学Stephen Boyd教授的convex optimization](http://stanford.edu/~boyd/cvxbook)），以Rosenbrock函数作为寻优的目标函数。

$f(x) = \displaystyle\sum_{i=1}^{N-1}100(x_i - x_{i-1}^2)^2 + (1 - x_{i-1})^2$

Rosenbrock函数的定义为：

In [8]:
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

#### Nelder-Mead单纯形法

单纯形法是运筹学中介绍的求解线性规划问题的通用方法，这里的Nelder-Mead单纯形法与其并不相同，只是用到单纯形的概念。设定起始点$x_0$，并进行最小化的寻优。`xatol`表示迭代收敛的容忍误差上界：

In [14]:
x_0 = np.array([0.5, 1.6, 1.1, 0.8, 1.2])
res = opt.minimize(rosen, x_0, method='nelder-mead', options={'xatol': 1e-8, 'disp': True})
print("Result of minimizing Rosenbrock function via Nelder-Mead Simplex algorithm:")
print(res)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 436
         Function evaluations: 706
Result of minimizing Rosenbrock function via Nelder-Mead Simplex algorithm:
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 1.6614969876635003e-17
             x: [ 1.000e+00  1.000e+00  1.000e+00  1.000e+00  1.000e+00]
           nit: 436
          nfev: 706
 final_simplex: (array([[ 1.000e+00,  1.000e+00, ...,  1.000e+00,
                         1.000e+00],
                       [ 1.000e+00,  1.000e+00, ...,  1.000e+00,
                         1.000e+00],
                       ...,
                       [ 1.000e+00,  1.000e+00, ...,  1.000e+00,
                         1.000e+00],
                       [ 1.000e+00,  1.000e+00, ...,  1.000e+00,
                         1.000e+00]]), array([ 1.661e-17,  6.321e-17,  7.441e-17,  8.244e-17,
                        9.532e-17,  1.079e-16

不使用函数的梯度，在略微复杂的情形下收敛速度比较慢。下面使用函数梯度进行寻优的方法。

#### Broyden-Fletcher-Goldfarb-Shanno法

Broyden-Fletcher-Goldfarb-Shanno（BFGS）法，先求一下Rosenbrock函数的梯度：

> 看不懂，先抄下来。

$\begin{equation}
\begin{split}
    \displaystyle\frac{∂f}{∂x_j} &= \displaystyle\sum_{i=1}^N200(x_i-x_{i-1}^2)(\delta_{i,j} - 2x_{i-1}\delta_{i-1,j}) - 2(1 - x_{i-1})\delta_{i-1,j}\\
    &= 200(x_j - x_{j-1}^2) - 400x_j(x_{j+1} - x_j^2) - 2(1 -x_j)
\end{split}
\end{equation}
$

其中，当 $i=j$ 时， $\delta_{i,j}=1$，否则 $\delta_{i,j}=0$。

边界的梯度是特例，有如下形式：

\begin{align}
    \frac{∂f}{∂x_0} &= -400x_0(x_1 - x_0^2) - 2(1 - x_0) \\
    \frac{∂f}{∂x_{N-1}} &= 200(x_{N-1} - x_{N-2}^2)
\end{align}

定义梯度向量的计算函数：

In [15]:
def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

梯度信息的引入在 `minimize` 函数中通过参数 `jac` 指定：

In [16]:
res = opt.minimize(rosen, x_0, method='BFGS', jac=rosen_der, options={'disp': True})
print("Result of minimizing Rosenbrock function via Broyden-Fletcher-Goldfarb-Shanno algorithm:")
print(res)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 39
         Function evaluations: 47
         Gradient evaluations: 47
Result of minimizing Rosenbrock function via Broyden-Fletcher-Goldfarb-Shanno algorithm:
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 1.569191726013783e-14
        x: [ 1.000e+00  1.000e+00  1.000e+00  1.000e+00  1.000e+00]
      nit: 39
      jac: [-3.604e-06  2.747e-06 -1.947e-07  2.784e-06 -1.410e-06]
 hess_inv: [[ 7.429e-03  1.251e-02 ...  4.698e-02  9.388e-02]
            [ 1.251e-02  2.506e-02 ...  9.443e-02  1.886e-01]
            ...
            [ 4.698e-02  9.443e-02 ...  3.786e-01  7.560e-01]
            [ 9.388e-02  1.886e-01 ...  7.560e-01  1.515e+00]]
     nfev: 47
     njev: 47


#### 牛顿共轭梯度法（Newton-Conjugate-Gradient algorithm）

用到梯度的方法还有牛顿法，牛顿法是收敛速度最快的方法，其缺点在于要求Hessian矩阵（二阶导数矩阵）。牛顿法大致的思路是采用泰勒展开的二阶近似。

（看不懂，所以内容略）

### 约束优化问题

（内容略）

详细内容见：https://www.kancloud.cn/wizardforcel/python-quant-uqer/186114 

省略内容待后续补充。